# infos

In [ ]:
# Version 1.2

# Modèle basé sur les 10 espèces les plus représentées du jeu de données.
# Images triées : Ok, choix possible
# Undersampling : Ok, choix possible
# Echantillonage : ok, choix possible

### Changelogs :

v1.2 :
- Ajout d'une fonction undersample pour équilibrer les observations
- La variable history (entrainement du modèle) est maintenant sauvegardée pour être exploitée dans d'autres notebooks
- Il est maintenant possible de définir en début de notebook :
    - Le csv utilisé (avec ou sans tri des images poubelles)
    - les données (full ou en echantillon avec choix de la taille)
    - L'undersampling (avec ou sans)

v1.1 :
- Nettoyage du code pour livraison projet
- Amélioration du modèle, entrainement sous format v1.1


v1.0 :
- Itération d'un modèle avec les images non triées et triées.
- Les modèles sont enregistrés sous format V1 pour tests ultérieurs

# Configuration Notebook

In [1]:
# Vérifier que les chemins soient correct avant toutes opérations

# Définition du dossiers contenant les images
chemin_images = '../../images/'

# Définition du fichier .csv utilisé (décommenter la ligne souhaitée)
chemin_csv = '../data/top10.csv'              # Les images sont triées
#chemin_csv = '../data/top10_no_tri.csv'      # Les images ne sont pas triées

# Dimensions retenues des images en entrée du modèle
img_dim = (200,200)
img_shape = (200,200,3)


# Choix des données d'entrainement :
pourcentage_echantillon = 0.1 # 0.1 représente 10% de la masse de données

# (Decommenter la ligne souhaitée)

# si 'oui', les données d'entrainement ne seront basées que sur un echantillon :
#set_echantillon = 'oui'
set_echantillon = 'non'

undersampling = 'oui'
#undersampling = 'non'

# Google Colab

/!\ Ignorer les cellules qui suivent si le notebook ne tourne pas sur Colab

In [ ]:
# Importer les images en format .zip
from google.colab import files
files.upload()

{}

In [2]:
# Monter le Drive si necessaire
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Dezipper le fichier images dans Colab
!unzip '/content/drive/MyDrive/SAS/images.zip' -d '/images'

Streaming output truncated to the last 5000 lines.
  inflating: /images/images/672562.jpg  
  inflating: /images/images/672563.jpg  
  inflating: /images/images/672564.jpg  
  inflating: /images/images/67257.jpg  
  inflating: /images/images/672570.jpg  
  inflating: /images/images/67258.jpg  
  inflating: /images/images/672580.jpg  
  inflating: /images/images/672584.jpg  
  inflating: /images/images/672585.jpg  
  inflating: /images/images/672586.jpg  
  inflating: /images/images/672587.jpg  
  inflating: /images/images/67259.jpg  
  inflating: /images/images/672602.jpg  
  inflating: /images/images/67261.jpg  
  inflating: /images/images/67264.jpg  
  inflating: /images/images/672656.jpg  
  inflating: /images/images/672657.jpg  
  inflating: /images/images/672664.jpg  
  inflating: /images/images/672665.jpg  
  inflating: /images/images/672666.jpg  
  inflating: /images/images/672667.jpg  
  inflating: /images/images/672668.jpg  
  inflating: /images/images/672703.jpg  
  inflating

In [4]:
# Remplacer les chemins en corrélation avec les dossiers colab

# Chemin vers le dossier images
chemin_images = '/images/images/'

# Chemin vers le fichier .csv utilisé
chemin_csv = '/content/drive/MyDrive/SAS/Jul23_bds_champignons/data/top10.csv'

# Librairies à charger

In [5]:
# Librairies générales servant dans le notebook
import pandas as pd
import os


# Librairies appelées pour l'utilisatio ndes fonctions définies dans la partie 'fonctions
from tensorflow.keras.applications.efficientnet import preprocess_input


# Librairies utilisées pour les callbacks
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from timeit import default_timer as timer
from tensorflow.keras.callbacks import TerminateOnNaN


# Librairies utilisées pour créer les pipelines et le mµodèle
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense


# Librairies utilisées pour la création des jeux d'entrainement, de test et de validation
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter


# Librairies utilisées pour l'évaluation du modèle une fois entrainé
import matplotlib.pyplot as plt
%matplotlib inline


# Librairies utilisées pour des sauvegardes et chargements de variables
import pickle

# Fonctions

In [6]:
def import_df(chemin_images, chemin_csv, pourcentage_echantillon = 0.1):
    '''
    Importe le fichier csv et construit 2 df :
        - Le DF basé sur le CSV original
        - Un DF echantillon comportant 10% de données aléatoires du DF original

    Arguments :
        - chemin_images : Chemin vers le dossier images
        - chemin_csv : Chemin vers le fichier .csv contenant les données utilisées
        pourcentage_echantillon : Taille du DF echantillon tiré du DF original
    '''


    # import du df
    df = pd.read_csv(chemin_csv, low_memory=False)
    df['image_url'] = df['image_url'].str.replace('.../images/', chemin_images)
    print(f"Nombre d'images chargées pour df: {df.shape[0]}")
    print(f"Nb especes dans df: {df['label'].nunique()}")


    # Contruction de l'echantillon
    L = len(df)
    L_ech = int(pourcentage_echantillon * L)
    df_ech = df.sample(n=L_ech, random_state=10)
    df_ech.reset_index(inplace=True, drop=True)
    print(f"Nombre d'images chargées pour df_ech: {df_ech.shape[0]}")
    print(f"Nb especes dans df_ech: {df_ech['label'].nunique()}")


    return df, df_ech

In [7]:
def undersampling_df(df, col):

    '''
    Undersample le df donné pour équilibrer le nombre d'pobservations par classe.
        - df : df à undersampler
        - col : colonne concernée par le GroupBy pour générer l'undersampling
    '''

    compte = df.groupby(col).count()
    min_samples = compte['image_url'].min()
    min_samples = int(min_samples)

    df_undersample = pd.DataFrame()

    for label, group in df.groupby('label'):
        df_undersample = pd.concat([df_undersample, group.sample(min_samples, replace=True)])
        df_undersample = df_undersample.reset_index(drop=True)

    return df_undersample

In [8]:
def augment_img(image_path, label):

   '''
    Modifie les images aléatoirement dans le dataset qui sera soumis au modèle.
      - image_path : URL des images (contenue dans la variable 'image_url' dans le DF chargé),
      - label : Variable contenant les classes
   '''

   # Lecture image, decodage
   img = tf.io.read_file(image_path)
   img = tf.image.decode_png(img, channels=3)

   # Redimensionnement selon dimensions définies en début de notebook (img_dim)
   img = tf.image.resize(img, img_dim)

   # Pre-processing pour transfert learning, modèle efficienNet
   img = preprocess_input(img)

   # Augmentations aléatoires des images :

   # Inversion Gauche/Droite
   img = tf.image.random_flip_left_right(img)

   # Inversion Haut/Bas
   img = tf.image.random_flip_up_down(img)

   # Modification luminosité
   img = tf.image.random_brightness(img, max_delta=0.2)

   # Modification contraste
   img = tf.image.random_contrast(img, lower=0.8, upper=1.2)

   # Conversion du type en float32
   img = tf.image.convert_image_dtype(img, tf.float32)

   # Normalisation
   img = (img - tf.math.reduce_min(img)) / (tf.math.reduce_max(img) - tf.math.reduce_min(img))

   return img, label

In [9]:
def create_tf_dataset(image_path, labels, batch_size):
    '''
    Créé un dataset Tensorflow selon les paramètres précisés.
        - image_path : chemin relatif de la variable contenant les images
        - labels : variable contenant les labels
        - batch_size : taille des batchs
    '''

    image_path = image_path.tolist()  # Convertir les chemins d'images en liste
    labels = labels.tolist()          # Convertir les labels en liste



    # Construction du Dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_path, labels))

    # .map appelle la fonction d'augmentation d'image définie
    dataset = dataset.map(augment_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Mélange aléatoire du dataset
    dataset = dataset.shuffle(buffer_size=len(image_path))

    # Découpage en batch
    dataset = dataset.batch(batch_size)

    # Optimisation : Charge les données en arrière-plan et maintien la charge CPU/GPU
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [10]:
def controle_presence_fichiers(df, chemin_images):

    '''
    Controle que les fichiers images soient bien présents sur le disque.
        - df : DataFrame contenant les url des fichiers images
        - chemin_images : Variable du DF contenant les url
    '''

    image_directory = chemin_images
    missing_files = []

    # Parcourir chaque ligne du DataFrame
    for index, row in df.iterrows():
        image_path = os.path.join(image_directory, row['image_lien'])

        if not os.path.exists(image_path):
            missing_files.append(image_path)

    # Afficher les fichiers non trouvés
    if missing_files:
        print("\nFichiers non trouvés :")
        for file_path in missing_files:
            print(file_path)

    # Ou préciser que tous les fichiers sont présents
    else:
        print("\nTous les fichiers sont présents.")

# Callbacks

Callback utilisés pour l'entrainement du modèle

### EarlyStopping

In [11]:
# earlystopping défini pour réduire le temps d'entrainement si l'accuracy n'évolue plus.
early_stopping = EarlyStopping(monitor = 'accuracy',
                               min_delta = 0.03,
                               patience = 8,
                               verbose = 1,
                               mode = 'auto',
                               restore_best_weights = True)

### Reduce LearningRate

In [12]:
# Reduction du learning Rate
reduceLR = ReduceLROnPlateau(monitor = 'val_loss',
                             min_delta = 0.01,
                             patience = 5,
                             factor = 0.15,
                             cooldown = 3,
                             verbose = 1)

### Checkpoint

In [13]:
# Enregistrement du modèle en cas d'amélioration lors de l'entrainement, ne conserve que la meilleure itération
checkpoint = ModelCheckpoint(filepath='../model/checkpoint_model', monitor='accuracy', save_best_only=True, verbose=1)

### Timer

In [14]:
# Suivi du temps d'entrainement
class TimingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.logs = []

    def on_epoch_begin(self, epoch, logs=None):
        self.starttime = timer()

    def on_epoch_end(self, epoch, logs=None):
        endtime = timer()
        elapsed_time = endtime - self.starttime
        self.logs.append(elapsed_time)
        print(f"Epoch {epoch + 1} took {elapsed_time:.2f} seconds")

time_callback = TimingCallback()

### Terminate on NaN

In [15]:
# Termine l'entrainement en cas de NaN
TON = TerminateOnNaN()

# Construction modèle

### Import du modèle pré-entrainé

In [16]:
# import du modèle efficientNetv2 pré-entrainé depuis Tensorflow Hub, les couches de convolution sont gelées
efficientNetv2 = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/classification/2"
pre_trained_model = hub.KerasLayer(efficientNetv2, input_shape=img_shape, trainable=False)

### Definition des couches

In [17]:
# Couches de convolution :
reshape_layer = layers.Reshape((1, 1, 1000))  # Cette couche permet d'entrer la sortie du modèle pré-entrainé dans la couche GlobalAveragePooling2D
gap = GlobalAveragePooling2D()

# Couches de regularisation :
dropout = Dropout(0.3)

# Couches dense :
dense = Dense(128, activation='relu', kernel_regularizer=l2(0.01)) # Régularisation L2
output = Dense(units = 10, activation='softmax') # Couche de sortie (10 classes)

### Construction du modèle

In [37]:
def build_model():

    model = tf.keras.Sequential()
    #model = models.Sequential()

    # Modèle pré-entrainé
    model.add(pre_trained_model)

    # Couches supplémentaires pour classification
    #model.add(reshape_layer)
    #model.add(gap)
    #model.add(dropout)
    model.add(dense)

    # Couche de sortie
    model.add(output)  # 10 classes de sortie

    return model

In [38]:
model = build_model()

### Compilation du modèle

In [39]:
# Compiler le modèle
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [40]:
# Afficher un résumé du modèle
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              7200312   
                                                                 
 dense (Dense)               (None, 128)               128128    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 7329730 (27.96 MB)
Trainable params: 129418 (505.54 KB)
Non-trainable params: 7200312 (27.47 MB)
_________________________________________________________________


# Pipeline Dataset

### Import des DataFrames

In [41]:
df, df_ech = import_df(chemin_images, chemin_csv, pourcentage_echantillon)

Nombre d'images chargées pour df: 60481
Nb especes dans df: 10
Nombre d'images chargées pour df_ech: 6048
Nb especes dans df_ech: 10


<ipython-input-6-b02316a4252b>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['image_url'] = df['image_url'].str.replace('.../images/', chemin_images)


In [42]:
# Préciser en début de notebook sur quelles données travaillées

if set_echantillon == 'oui':
  donnees_training = df_ech
else:
  donnees_training = df

In [43]:
# Controle de la présence des fichiers images
controle_presence_fichiers(donnees_training, chemin_images)

# On supprime ensuite la colonne image_lien qui ne sert qu'à controler la présence des fichiers.
donnees_training.drop('image_lien', axis=1, inplace=True)


Tous les fichiers sont présents.


In [44]:
donnees_training.head()

,label,image_url
0,Agaricales,../images/images/486562.jpg
1,Agaricales,../images/images/509189.jpg
2,Agaricales,../images/images/486561.jpg
3,Agaricales,../images/images/508881.jpg
4,Agaricales,../images/images/230787.jpg


In [45]:
# undersampling des classes : (si défini)
if undersampling == 'oui':
  donnees_training = undersampling_df(donnees_training, col = 'label')
  print(donnees_training.groupby('label').count())

else:
  print("Pas d'undersampling programmé, les données sont déséquilibrées.")

             image_url
label                 
Agaricales        3918
Agaricus          3918
Amanita           3918
Cortinarius       3918
Entoloma          3918
Inocybe           3918
Mycena            3918
Polyporales       3918
Psathyrella       3918
Russula           3918


### Construction des jeux de données (train, test et validation)

In [46]:
data = donnees_training.drop('label', axis=1)
target = donnees_training['label']

 # Encodage de la variable 'label'
s = LabelEncoder()
target = s.fit_transform(target)

# On construit le jeu d'entrainnement. X_temp et y_temps servent pour la construction des jeux de test et validation
X_train, X_temp, y_train, y_temp = train_test_split(data, target, test_size=0.20, random_state=10)

# On split les temp en 50% pour test, 50% pour validation
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=10)

### Construction des dataset Tensorflow

In [47]:
batch_size = 64
# Les datasets sont créés à partir de la fonction create_tf_dataset définie dans la partie 'Fonctions'
ds_train= create_tf_dataset(X_train.image_url, y_train, batch_size)
ds_val = create_tf_dataset(X_val.image_url, y_val, batch_size)

# Entrainement du modèle

### Methode .fit

In [48]:
history = model.fit(ds_train,
                    validation_data = ds_val,
                    epochs=30,
                    callbacks = [early_stopping, reduceLR, checkpoint, time_callback, TON],
                    batch_size = batch_size,
                    verbose=True)

Epoch 1/30
489/490 [============================>.] - ETA: 0s - loss: 1.3767 - accuracy: 0.5756
Epoch 1: accuracy did not improve from 0.61773
Epoch 1 took 38.54 seconds
490/490 [==============================] - 39s 49ms/step - loss: 1.3768 - accuracy: 0.5755 - val_loss: 1.3292 - val_accuracy: 0.5944 - lr: 0.0010
Epoch 2/30
489/490 [============================>.] - ETA: 0s - loss: 1.3366 - accuracy: 0.5919
Epoch 2: accuracy did not improve from 0.61773
Epoch 2 took 34.06 seconds
490/490 [==============================] - 34s 46ms/step - loss: 1.3364 - accuracy: 0.5918 - val_loss: 1.3029 - val_accuracy: 0.6008 - lr: 0.0010
Epoch 3/30
489/490 [============================>.] - ETA: 0s - loss: 1.3266 - accuracy: 0.5925
Epoch 3: accuracy did not improve from 0.61773
Epoch 3 took 34.03 seconds
490/490 [==============================] - 34s 46ms/step - loss: 1.3269 - accuracy: 0.5924 - val_loss: 1.3357 - val_accuracy: 0.5858 - lr: 0.0010
Epoch 4/30
489/490 [============================>.] 

### [En cas de crash durant l'entrainement]

In [ ]:
# Cellule reservée au chargement du modèle checkpoint pour relancer l'entrainement si le kernel crash
# /!\ Ne pas utiliser ces cellules si l'entrainement s'est réalisé en entier /!\
from tensorflow.keras.models import load_model
loaded_model = load_model('../model/checkpoint_model')
epoch_crash = 11            # Préciser le dernier epoch interrompu
nb_epochs_a_realiser = 10   # Nombre total d'epochs souhaité à partir de la reprise

In [ ]:
# Reprise de l'entrainement
history = loaded_model.fit(ds_train,
                           validation_data=ds_val,
                           initial_epoch=epoch_crash,
                           epochs=nb_epochs_a_realiser,
                           callbacks=[tensorboard, early_stopping, reduceLR, checkpoint, time_callback, TON],
                           verbose=True)

# Sauvegarde du modèle

In [49]:
# Changer le nom du modèle si il s'agit d'un nouvel entrainement

# Save en dur ou sur Gdrive
#nom_modele = '../model/gpot_v02b_full_tri_undersampled'                          # Save en dur
nom_modele =  '/content/drive/MyDrive/SAS/model/gpot_v02b_full_tri_undersampled'  # Save Gdrive


# historique d'entrainement
history_gpot_v02_full_tri_undersampled = history

# Save en dur
# chemin_acces = '../history/history_gpot_v02b_full_tri_undersampled.pkl'

# Save sur GDrive
chemin_acces ='/content/drive/MyDrive/SAS/history/history_gpot_v02b_full_tri_undersampled.pkl'


In [50]:
model.save(nom_modele)

with open(chemin_acces, 'wb') as file:
    pickle.dump(history_gpot_v02_full_tri_undersampled, file)